### 임베딩 모델 사용

##### 0. Blob 스토리지 생성 후, CORS 설정
##### 1. Blob 스토리지에 CSV 파일 업로드
##### 2. OpenAI
- 채팅 플레이 그라운드 '배포'탭 -> 모델 배포 -> 'text-embedding-ada-002' 선택 -> '사용자 정의'하여 분당 토큰 조절 -> 배포
##### 3. AI Search
- '개요' -> 데이터 가져오기(신규) -> Blob Storage -> RAG -> 스토리지 및 컨테이너 선택 -> OpenAI 리소스 및 text-embedding 모델 선택 -> 만들기 (인덱스가 생김)
##### 4. OpenAI
- 채팅 플레이 그라운드 - 데이터 추가 - 'Azure AI 검색' - 3번에 생성된 인덱스 선택 - '벡터 검색' 체크 - 임베딩 모델 선택 - 검색 유형 '하이브리드 + 의미체계', 기존 의미 체계 검색 구성 선택 - API 키 선택 
##### * Blob 스토리지에 데이터 추가 시, AI Search의 인덱서(인덱스X)에서, 초기화 및 재실행하여 추가된 데이터 기반 검색 가능

In [ ]:
import os
import base64
from openai import AzureOpenAI
from dotenv import load_dotenv

# .env 환경변수 로드
load_dotenv()
endpoint = os.getenv("ENDPOINT_URL", "https://8ai022-openai.openai.azure.com/")                     # OpenAI Endpoint
deployment = os.getenv("AZURE_OAI_DEPLOYMENT", "gpt-4o-mini")                                       # OpenAI Model Name
subscription_key = os.getenv("AZURE_OAI_KEY", "REPLACE_WITH_YOUR_KEY_VALUE_HERE")                   # OpenAI Key
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://8ai022-ai-search.search.windows.net")       # AI-Search Endpoint
search_key = os.getenv("AZURE_SEARCH_KEY", "put your Azure AI Search admin key here")               # AI-Search Key
search_index = "rag-movie"

# Initialize Azure OpenAI client with key-based authentication
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2025-01-01-preview",
)

# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')

# Prepare the chat prompt
chat_prompt = [
    {
        "role": "system",
        "content": "주인님이 원하는 결과를 충실하게 답변하도록! 극극존칭 사용 필수!"
    },
    {
        "role": "user",
        "content": "INTJ가 좋아할만한 영화들 추천해줘"
    }
]

# Include speech result if speech is enabled
messages = chat_prompt

# Generate the completion
completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    max_tokens=6553,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False,
    extra_body={
      "data_sources": [{
          "type": "azure_search",
          "parameters": {
            "endpoint": f"{search_endpoint}",
            "index_name": f"{search_index}",
            "semantic_configuration": "rag-movie-semantic-configuration",
            "query_type": "vector_semantic_hybrid", # 하이브리드 + 의미체계
            "fields_mapping": {},
            "in_scope": True,
            #"role_information": "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다.", #상단 chat_prompt에서 system role 작성했으므로, 지워주어야 함
            "filter": None,
            "strictness": 4,        # 엄격성
            "top_n_documents": 5,   # 검색된 문서
            "authentication": {
              "type": "api_key",
              "key": f"{search_key}"
            },
            "embedding_dependency": {
              "type": "deployment_name",
              "deployment_name": "text-embedding-ada-002"   # 텍스트 임베딩 모델
            }
          }
        }]
    }
)

print(completion.to_json())
print(completion.choices[0].message.context['citations'])
    

{
  "id": "1d10a4a5-c126-454c-8bfd-70dbb24d027e",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "주인님, INTJ 성향을 가진 분들이 선호할만한 영화들을 추천해 드리겠습니다. INTJ는 분석적이고 전략적인 사고를 선호하며, 깊이 있는 스토리와 복잡한 캐릭터를 좋아하는 경향이 있습니다. 다음은 INTJ가 좋아할 만한 영화 목록입니다:\n\n1. **인셉션 (Inception)** - 꿈과 현실의 경계를 탐구하는 복잡한 플롯과 철학적 질문들이 돋보이는 영화입니다.\n2. **파이트 클럽 (Fight Club)** - 현대 사회에 대한 비판과 정체성 탐구가 주된 주제로, INTJ의 분석적 사고를 자극할 수 있는 작품입니다.\n3. **셜록 홈즈 (Sherlock Holmes)** - 추리와 논리적인 사고가 중요한 요소로 작용하는 이 영화는 INTJ의 탐구심을 자극할 것입니다.\n4. **더 소셜 네트워크 (The Social Network)** - 페이스북의 탄생 과정을 다룬 이 영화는 기업가 정신과 인간 관계의 복잡성을 잘 보여줍니다.\n5. **블레이드 러너 (Blade Runner)** - 인간성과 인공지능에 대한 깊은 질문을 던지는 SF 영화로, 철학적인 사유를 자극합니다.\n\n이 외에도 INTJ 분들은 스릴러, 드라마, 과학 소설 장르의 영화에서 매력을 느낄 수 있습니다. 더 많은 추천이 필요하시다면 언제든지 말씀해 주세요!",
        "role": "assistant",
        "end_turn": true,
        "context": {
          "citations": [
            {
              "content": "백수 남편과 삼촌, 군생활에 적응 못한 아들, 반항적인 딸, 이 아슬아슬한 가족의 내일은 무사할